# reasoning_bank

> ReasoningBank: Procedural recipes for ontology exploration with RLM

In [ ]:
#| default_exp reasoning_bank

## Overview

The ReasoningBank provides **ontology-specific procedural recipes** that guide the LLM in domain-specific ontology exploration tasks.

### Architecture: Memory vs Recipes

**IMPORTANT ARCHITECTURAL DECISION (2026-01-19 Refactor):**

This module was refactored to align with the ReasoningBank paper's memory-based learning model:

- **General strategies** (universal patterns): → `procedural_memory.bootstrap_general_strategies()` (LEARNED)
- **Ontology-specific patterns** (PROV, SIO, etc.): → `reasoning_bank.ONTOLOGY_RECIPES` (AUTHORED)

**Rationale:**
- General strategies are LEARNED from experience and stored as MemoryItems (BM25-retrieved)
- Ontology-specific patterns are AUTHORED by domain experts and stored as Recipes (always injected)
- This separation enables: learning new strategies over time while maintaining domain-specific guidance

### Four-Layer Context Injection

1. **Layer 0: Sense Card** - Compact ontology metadata (always injected)
2. **Layer 1: Retrieved Memories** - General strategies from procedural_memory (BM25-retrieved)
3. **Layer 2: Ontology Recipes** - Domain-specific patterns (this module)
4. **Layer 3: Base Context** - GraphMeta summary

### Design Principles

- **Explicit procedures**: Step-by-step tool usage patterns
- **Grounded in tools**: Only reference available functions
- **Expected iterations**: Set clear convergence expectations
- **Domain-specific**: Recipes capture ontology-specific conventions (e.g., PROV Activity-Entity patterns)

## Imports

In [ ]:
#| export
from dataclasses import dataclass
from typing import Optional

## Recipe Schema

In [ ]:
#| export
@dataclass
class Recipe:
    """A procedural recipe for ontology exploration.
    
    Recipes provide explicit step-by-step guidance on HOW to use ontology
    exploration tools to accomplish specific tasks.
    """
    id: str  # Unique identifier (e.g., 'recipe-1-describe-entity')
    title: str  # Human-readable title
    when_to_use: str  # When this recipe applies
    procedure: str  # Step-by-step markdown checklist
    expected_iterations: int  # Expected number of iterations to converge
    layer: int  # 0=sense, 1=core, 2=task-type, 3=ontology-specific
    task_types: list[str]  # Which task types this applies to
    ontology: Optional[str] = None  # None = universal, else ontology name
    
    def format_for_injection(self) -> str:
        """Format recipe for context injection."""
        lines = [
            f"### Recipe: {self.title}",
            "",
            f"**When to use:** {self.when_to_use}",
            "",
            f"**Expected iterations:** {self.expected_iterations}",
            "",
            "**Procedure:**",
            self.procedure
        ]
        return '\n'.join(lines)

## Ontology-Specific Recipes (Layer 2)

**After 2026-01-19 Refactor:** This section now contains ONLY ontology-specific recipes.

**What moved:** Universal patterns (describe entity, find subclasses, etc.) moved to `procedural_memory.bootstrap_general_strategies()`.

**What stays:** Ontology-specific patterns like:
- PROV: Activity-Entity relationship patterns
- SIO: Measurement and process patterns  
- Domain-specific conventions and idioms

**Current status:** Placeholder (ONTOLOGY_RECIPES = []) - to be populated with domain-specific recipes as needed.

In [ ]:
#| export
# Ontology-specific recipes only
# General strategies are now in procedural_memory.bootstrap_general_strategies()

ONTOLOGY_RECIPES = [
    # Placeholder for ontology-specific patterns
    # Example:
    # Recipe(
    #     id='prov-activity-entity-pattern',
    #     title='Activity-Entity Relationships in PROV',
    #     ontology='prov',  # PROV-specific
    #     procedure='In PROV, Activities relate to Entities via...'
    # )
]


In [ ]:
# Test Recipe creation
test_recipe = CORE_RECIPES[1]  # recipe-1-describe-entity
print(f"Recipe: {test_recipe.title}")
print(f"Layer: {test_recipe.layer}")
print(f"Expected iterations: {test_recipe.expected_iterations}")
print(f"Task types: {test_recipe.task_types}")
print(f"\nFormatted:")
print(test_recipe.format_for_injection())

## Recipe Retrieval Functions

In [ ]:
#| export
def classify_task_type(query: str) -> str:
    """Classify query into task type for recipe selection.
    
    Args:
        query: User query string
        
    Returns:
        Task type: 'entity_discovery', 'entity_description', 'hierarchy',
                  'property_discovery', 'pattern_search', 'relationship_discovery'
    """
    query_lower = query.lower()
    
    # Hierarchy queries
    if any(word in query_lower for word in ['subclass', 'superclass', 'parent', 'child', 'hierarchy']):
        return 'hierarchy'
    
    # Property queries
    if any(word in query_lower for word in ['property', 'properties', 'domain', 'range', 'relates']):
        return 'property_discovery'
    
    # Relationship queries
    if any(word in query_lower for word in ['path', 'connects', 'relationship', 'related']):
        return 'relationship_discovery'
    
    # Pattern search
    if any(word in query_lower for word in ['find all', 'search for', 'list', 'matching']):
        return 'pattern_search'
    
    # Entity description ("What is X?")
    if any(word in query_lower for word in ['what is', 'describe', 'tell me about']):
        return 'entity_description'
    
    # Default to entity discovery
    return 'entity_discovery'


def get_core_recipes(limit: int = 2) -> list[Recipe]:
    """Get always-injected core recipes.
    
    Args:
        limit: Maximum number of core recipes to return
        
    Returns:
        List of core Recipe objects
    """
    # Always include recipe-0 (How to Use Sense Data)
    core = [CORE_RECIPES[0]]  # recipe-0-use-sense
    
    # Add most common recipe (describe entity)
    if limit > 1:
        core.append(CORE_RECIPES[1])  # recipe-1-describe-entity
    
    return core[:limit]


def retrieve_task_recipes(task_type: str, k: int = 2) -> list[Recipe]:
    """Retrieve task-type specific recipes.
    
    Args:
        task_type: Task type from classify_task_type()
        k: Number of recipes to retrieve
        
    Returns:
        List of relevant Recipe objects
    """
    # Filter recipes by task type
    relevant = [
        recipe for recipe in CORE_RECIPES
        if task_type in recipe.task_types or 'all' in recipe.task_types
    ]
    
    # Exclude recipe-0 (always injected separately)
    relevant = [r for r in relevant if r.id != 'recipe-0-use-sense']
    
    # Sort by expected iterations (prefer faster recipes)
    relevant.sort(key=lambda r: r.expected_iterations)
    
    return relevant[:k]


def format_recipes_for_injection(recipes: list[Recipe]) -> str:
    """Format recipes as markdown for context injection.
    
    Args:
        recipes: List of Recipe objects
        
    Returns:
        Formatted markdown string
    """
    if not recipes:
        return ""
    
    sections = ["## Procedural Recipes", ""]
    
    for recipe in recipes:
        sections.append(recipe.format_for_injection())
        sections.append("")  # Blank line between recipes
    
    return '\n'.join(sections)

In [ ]:
# Test recipe retrieval
print("Test 1: Classify task types")
queries = [
    "What is Activity?",
    "Find all subclasses of Activity",
    "What properties have Entity as domain?",
    "How are Activity and Entity related?"
]

for q in queries:
    task_type = classify_task_type(q)
    print(f"  '{q}' → {task_type}")

print("\nTest 2: Get core recipes")
core = get_core_recipes(limit=2)
print(f"  Retrieved {len(core)} core recipes:")
for r in core:
    print(f"    - {r.title}")

print("\nTest 3: Retrieve task-specific recipes")
task_recipes = retrieve_task_recipes('hierarchy', k=2)
print(f"  Retrieved {len(task_recipes)} recipes for 'hierarchy':")
for r in task_recipes:
    print(f"    - {r.title}")

print("\nTest 4: Format recipes")
formatted = format_recipes_for_injection(core[:1])
print(f"  Formatted length: {len(formatted)} chars")
print(f"\nFormatted output (first 300 chars):")
print(formatted[:300] + "...")

## Context Injection

Four-layer context injection strategy:
1. **Layer 0:** Sense card (from structured sense)
2. **Layer 1:** Core recipes (always injected)
3. **Layer 2:** Task-type recipes (query-dependent)
4. **Layer 3:** Ontology-specific knowledge (future)

In [ ]:
#| export
def inject_context(
    query: str,
    base_context: str,
    sense: dict = None,
    memory_store = None,
    ontology: str = None,
    max_memories: int = 3,
    max_task_recipes: int = 2
) -> str:
    """Build complete context with sense + memories + recipes.
    
    Injection order:
    1. Sense card (Layer 0) - if provided
    2. Retrieved memories (Layer 1) - general strategies from memory_store
    3. Task-type recipes (Layer 2) - ontology-specific patterns
    4. Base context - original graph summary
    
    Args:
        query: User query
        base_context: Base context string (e.g., GraphMeta.summary())
        sense: Structured sense document (from build_sense_structured)
        memory_store: MemoryStore with general strategies
        ontology: Optional ontology name for ontology-specific recipes
        max_memories: Maximum memories to retrieve
        max_task_recipes: Maximum task-type recipes to inject
        
    Returns:
        Enhanced context string
    """
    from rlm.ontology import get_sense_context
    
    sections = []
    
    # Layer 0: Sense card (if provided)
    if sense:
        sense_ctx = get_sense_context(query, sense)
        sections.append(sense_ctx)
    
    # Layer 1: Retrieved memories (general strategies)
    if memory_store:
        from rlm.procedural_memory import retrieve_memories, format_memories_for_injection
        memories = retrieve_memories(memory_store, query, k=max_memories)
        if memories:
            sections.append(format_memories_for_injection(memories))
    
    # Layer 2: Ontology-specific recipes (if any)
    if ontology and ONTOLOGY_RECIPES:
        ontology_recipes = [
            r for r in ONTOLOGY_RECIPES
            if r.ontology == ontology or r.ontology is None
        ]
        if ontology_recipes:
            sections.append(format_recipes_for_injection(ontology_recipes[:max_task_recipes]))
    
    # Layer 3: Base context (original)
    if base_context:
        sections.append("## Base Context")
        sections.append("")
        sections.append(base_context)
    
    return '\n\n'.join(sections)


## Enhanced RLM Runner

In [ ]:
#| export
def rlm_run_enhanced(
    query: str,
    context: str,
    ns: dict = None,
    sense: dict = None,
    memory_store = None,
    ontology: str = None,
    **kwargs
) -> tuple:
    """RLM run with sense and memory injection.
    
    This is a drop-in replacement for rlm_run() that automatically
    enhances context with:
    - Layer 0: Structured sense card
    - Layer 1: Retrieved procedural memories (general strategies)
    - Layer 2: Ontology-specific recipes
    
    Args:
        query: User query
        context: Base context (e.g., GraphMeta.summary())
        ns: Namespace dict
        sense: Structured sense document (from build_sense_structured)
        memory_store: MemoryStore with general strategies
        ontology: Optional ontology name
        **kwargs: Additional arguments passed to rlm_run()
        
    Returns:
        (answer, iterations, final_ns) - same as rlm_run()
    """
    from rlm.core import rlm_run
    
    # Build enhanced context with layered injection
    enhanced_context = inject_context(
        query=query,
        base_context=context,
        sense=sense,
        memory_store=memory_store,
        ontology=ontology
    )
    
    # Call base rlm_run with enhanced context
    return rlm_run(query, enhanced_context, ns=ns, **kwargs)


## Tests

In [ ]:
# Test inject_context with memory_store
from rlm.procedural_memory import MemoryStore, bootstrap_general_strategies

print("Test: inject_context() with memory_store")
print("=" * 70)

# Create memory store with general strategies
memory_store = MemoryStore()
strategies = bootstrap_general_strategies()
for s in strategies:
    memory_store.add(s)

print(f"Memory store: {len(memory_store.memories)} strategies")

# Simulate a sense document
test_sense = {
    'sense_card': {
        'ontology_id': 'test',
        'domain_scope': 'Test ontology',
        'triple_count': 100,
        'class_count': 10,
        'property_count': 5,
        'key_classes': [],
        'key_properties': [],
        'label_predicates': ['rdfs:label'],
        'description_predicates': ['rdfs:comment'],
        'available_indexes': {},
        'quick_hints': ['Test hint'],
        'uri_pattern': 'http://test.org/'
    },
    'sense_brief': {}
}

query = "What is Activity?"
base_context = "Test base context"

enhanced = inject_context(
    query=query,
    base_context=base_context,
    sense=test_sense,
    memory_store=memory_store,
    max_memories=2
)

print(f"\nQuery: '{query}'")
print(f"\nEnhanced context length: {len(enhanced)} chars")
print(f"\nContext structure:")
if '# Ontology:' in enhanced:
    print("  ✓ Layer 0: Sense card")
if 'Relevant Prior Experience' in enhanced or 'Describe Entity' in enhanced:
    print("  ✓ Layer 1: Retrieved memories (general strategies)")
if 'Base Context' in enhanced:
    print("  ✓ Base context included")

print(f"\nFirst 400 chars of enhanced context:")
print("-" * 70)
print(enhanced[:400] + "...")


## Validation

Validate memory-recipe separation and ensure correct architecture.

In [ ]:
#| export
def validate_memory_recipe_separation(memory_store) -> dict:
    """Ensure general strategies aren't duplicated in ONTOLOGY_RECIPES.
    
    Validates:
    - No title overlap between universal memories and ontology recipes
    - All ONTOLOGY_RECIPES have ontology field set (domain-specific)
    
    Args:
        memory_store: MemoryStore (typically with bootstrap strategies)
    
    Returns:
        Dictionary with validation results
    """
    # Get titles from memory store (bootstrap strategies)
    memory_titles = set(
        m.title.lower() 
        for m in memory_store.memories 
        if 'universal' in m.tags
    )
    
    # Get titles from ONTOLOGY_RECIPES
    recipe_titles = set(r.title.lower() for r in ONTOLOGY_RECIPES)
    
    # Check for overlap
    overlap = memory_titles & recipe_titles
    
    # Check ONTOLOGY_RECIPES are truly ontology-specific
    ontology_specific = all(
        r.ontology is not None
        for r in ONTOLOGY_RECIPES
    ) if ONTOLOGY_RECIPES else True  # Empty is valid
    
    return {
        'valid': len(overlap) == 0 and ontology_specific,
        'overlap_count': len(overlap),
        'overlapping_titles': list(overlap),
        'all_recipes_have_ontology': ontology_specific,
        'ontology_recipes_count': len(ONTOLOGY_RECIPES)
    }


In [ ]:
# Test memory-recipe separation validation
from rlm.procedural_memory import MemoryStore, bootstrap_general_strategies

print("Test: validate_memory_recipe_separation()")
print("=" * 60)

memory_store = MemoryStore()
for s in bootstrap_general_strategies():
    memory_store.add(s)

result = validate_memory_recipe_separation(memory_store)
print(f"Valid: {result['valid']}")
print(f"Overlap count: {result['overlap_count']}")
print(f"Ontology recipes count: {result['ontology_recipes_count']}")
print(f"All recipes have ontology field: {result['all_recipes_have_ontology']}")

if result['valid']:
    print("\n✓ Memory-recipe separation validated successfully")
else:
    print(f"\n✗ Validation failed: {result['overlapping_titles']}")
